<a href="https://colab.research.google.com/github/jyoti246/Compilers_lab/blob/master/Word_Frequency_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import csv
from scipy import stats
import string
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# t_web.csv available here, place it in drive and just run each cell
# https://drive.google.com/file/d/1e218lAzD9pBsSeFLlXHz3Kh11hI2lDqa/view?usp=sharing
df_bible = pd.read_csv('/content/drive/MyDrive/t_web.csv')
print(df_bible)
bible_text = ""
for i in range(len(df_bible)):
    bible_text = bible_text + df_bible['t'][i] + " "

             id   b   c   v                                                  t
0       1001001   1   1   1  In the beginning God{After "God," the Hebrew h...
1       1001002   1   1   2  Now the earth was formless and empty. Darkness...
2       1001003   1   1   3  God said, "Let there be light," and there was ...
3       1001004   1   1   4  God saw the light, and saw that it was good. G...
4       1001005   1   1   5  God called the light Day, and the darkness he ...
...         ...  ..  ..  ..                                                ...
31097  66022017  66  22  17  The Spirit and the bride say, "Come!" He who h...
31098  66022018  66  22  18  I testify to everyone who hears the words of t...
31099  66022019  66  22  19  If anyone takes away from the words of the boo...
31100  66022020  66  22  20  He who testifies these things says, "Yes, I co...
31101  66022021  66  22  21  The grace of the Lord Jesus Christ be with all...

[31102 rows x 5 columns]


In [ ]:
TRAIN_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/train/lcp_single_train.tsv"
TEST_DATAPATH = "https://raw.githubusercontent.com/MMU-TDMLab/CompLex/master/test-labels/lcp_single_test.tsv"
df_train = pd.read_csv(TRAIN_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
df_test = pd.read_csv(TEST_DATAPATH, sep = '\t', quotechar="'", quoting = csv.QUOTE_NONE)
df_train['complexity'] = df_train['complexity'].astype(float)
df_test['complexity'] = df_test['complexity'].astype(float)

In [ ]:
train_text = ""
test_text = ""
for i in range(len(df_train)):
    train_text = train_text + df_train['sentence'][i] + " "
for i in range(len(df_test)):
    test_text = test_text + df_test['sentence'][i] + " "
print(train_text)

Behold, there came up out of the river seven cattle, sleek and fat, and they fed in the marsh grass. I am a fellow bondservant with you and with your brothers, the prophets, and with those who keep the words of this book. The man, the lord of the land, said to us, 'By this I will know that you are honest men: leave one of your brothers with me, and take grain for the famine of your houses, and go your way. Shimei had sixteen sons and six daughters; but his brothers didn't have many children, neither did all their family multiply like the children of Judah. "He has put my brothers far from me. Therefore I urge you, brothers, by the mercies of God, to present your bodies a living sacrifice, holy, acceptable to God, which is your spiritual service. Rehoboam loved Maacah the daughter of Absalom above all his wives and his concubines: (for he took eighteen wives, and sixty concubines, and became the father of twenty-eight sons and sixty daughters.) Then they brought out the king's son, and 

In [ ]:
exclude = set(string.punctuation)
device = 'cuda'


In [ ]:
train_text = ''.join(ch for ch in train_text if ch not in exclude)
test_text = ''.join(ch for ch in test_text if ch not in exclude)
bible_text = ''.join(ch for ch in bible_text if ch not in exclude)
train_text = train_text.lower()
test_text = test_text.lower()
bible_text = bible_text.lower()
whole_text = train_text+' '+test_text+' '+bible_text
# getting frequency of each word in whole set we got from whole bible and that available in train and test
whole_set = Counter(whole_text.split())
# train_set = Counter(train_text.split())
# test_set = Counter(test_text.split())

In [ ]:
train_input = []
test_input = []
for i in range(len(df_train)):
    input = [0 for _ in range(8)]
    x = str(df_train['sentence'][i])
    x = ''.join(ch for ch in x if ch not in exclude)
    x = x.lower()
    x = x.split()
    token = str(df_train['token'][i])
    token = ''.join(ch for ch in token if ch not in exclude)
    token = token.lower()
    for j in range(len(x)):
        if x[j] == token:
            # input[2] += whole_set[x[j]]
            # if j >= 6 and whole_set[x[j - 4]] > 0:
            #     input[0] += whole_set[x[j - 4]]/(whole_set[x[j]]+1)
            # if j >= 5 and whole_set[x[j - 4]] > 0:
            #     input[0] += whole_set[x[j - 4]]/(whole_set[x[j]]+1)
            if j >= 4 and whole_set[x[j - 4]] > 0:
                input[0] += whole_set[x[j - 4]]/(whole_set[x[j]]+1)
            if j >= 3 and whole_set[x[j - 3]] > 0:
                input[1] += whole_set[x[j - 3]]/(whole_set[x[j]]+1)
            if j >= 2 and whole_set[x[j - 2]] > 0:
                input[2] += whole_set[x[j - 2]]/(whole_set[x[j]]+1)
            if j >= 1 and whole_set[x[j - 1]] > 0:
                input[3] += whole_set[x[j - 1]]/(whole_set[x[j]]+1)

            if j < len(x) - 1 and whole_set[x[j + 1]] > 0:
                input[4] += whole_set[x[j + 1]]/(whole_set[x[j]]+1)
            if j < len(x) - 2 and whole_set[x[j + 2]] > 0:
                input[5] += whole_set[x[j + 2]]/(whole_set[x[j]]+1)
            if j < len(x) - 3 and whole_set[x[j + 3]] > 0:
                input[6] += whole_set[x[j + 3]]/(whole_set[x[j]]+1)
            if j < len(x) - 4 and whole_set[x[j + 4]] > 0:
                input[7] += whole_set[x[j + 4]]/(whole_set[x[j]]+1)
    # adding extra features which are multiplication of above features to get better results
    prod = 1.0
    for i in range(8):
        prod *= input[i]
        for j in range(8):
            input.append((input[i]*input[j])**(1/2))
    input.append(prod**(1/8))
    train_input.append(input)
for i in range(len(df_test)):
    input = [0 for _ in range(8)]
    x = str(df_test['sentence'][i])
    x = ''.join(ch for ch in x if ch not in exclude)
    x = x.lower()
    x = x.split()
    token = str(df_test['token'][i])
    token = ''.join(ch for ch in token if ch not in exclude)
    token = token.lower()
    for j in range(len(x)):
        if x[j] == token:
            # input[2] += whole_set[x[j]]
            if j >= 4 and whole_set[x[j - 4]] > 0:
                input[0] += whole_set[x[j - 4]]/(whole_set[x[j]]+1)
            if j >= 3 and whole_set[x[j - 3]] > 0:
                input[1] += whole_set[x[j - 3]]/(whole_set[x[j]]+1)
            if j >= 2 and whole_set[x[j - 2]] > 0:
                input[2] += whole_set[x[j - 2]]/(whole_set[x[j]]+1)
            if j >= 1 and whole_set[x[j - 1]] > 0:
                input[3] += whole_set[x[j - 1]]/(whole_set[x[j]]+1)

            if j < len(x) - 1 and whole_set[x[j + 1]] > 0:
                input[4] += whole_set[x[j + 1]]/(whole_set[x[j]]+1)
            if j < len(x) - 2 and whole_set[x[j + 2]] > 0:
                input[5] += whole_set[x[j + 2]]/(whole_set[x[j]]+1)
            if j < len(x) - 3 and whole_set[x[j + 3]] > 0:
                input[6] += whole_set[x[j + 3]]/(whole_set[x[j]]+1)
            if j < len(x) - 4 and whole_set[x[j + 4]] > 0:
                input[7] += whole_set[x[j + 4]]/(whole_set[x[j]]+1)

    # adding extra features which are multiplication of above features to get better results
    prod = 1.0
    for i in range(8):
        prod *= input[i]
        for j in range(8):
            input.append((input[i]*input[j])**(1/2))
    input.append(prod**(1/8))
    test_input.append(input)

In [ ]:
# Normalize the input tensors, that is, train_input and test_input

In [ ]:
labels = [i for i in df_train['complexity']]
test_labels = [i for i in df_test['complexity']]

In [ ]:
class NN(nn.Module):
  def __init__(self, input_dim):
    super().__init__()
    self.linear1 = nn.Linear(input_dim, 13850)
    self.linear2 = nn.Linear(13850, 128)
    self.linear3 = nn.Linear(128, 256)
    self.linear4 = nn.Linear(256, 256)
    self.linear5 = nn.Linear(256, 128)
    self.linear6 = nn.Linear(128, 64)
    self.linear7 = nn.Linear(64, 1)
    # self.linear1.weight.data.fill_(1)
    # self.linear1.bias.data.fill_(1)
    # self.linear2.weight.data.fill_(1)
    # self.linear2.bias.data.fill_(1)
    # self.linear3.weight.data.fill_(1)
    # self.linear3.bias.data.fill_(1)
    # self.linear4.weight.data.fill_(1)
    # self.linear4.bias.data.fill_(1)
    # self.linear5.weight.data.fill_(1)
    # self.linear5.bias.data.fill_(1)
    # self.linear6.weight.data.fill_(1)
    # self.linear6.bias.data.fill_(1)
    # self.linear7.weight.data.fill_(1)
    # self.linear7.bias.data.fill_(1)
    
    # torch.nn.init.xavier_uniform_(self.linear1.weight)
    # torch.nn.init.xavier_uniform_(self.linear2.weight)
    # torch.nn.init.xavier_uniform_(self.linear3.weight)
    # torch.nn.init.xavier_uniform_(self.linear4.weight)
    # torch.nn.init.xavier_uniform_(self.linear5.weight)
    # torch.nn.init.xavier_uniform_(self.linear6.weight)
    # torch.nn.init.xavier_uniform_(self.linear7.weight)
    # torch.nn.init.xavier_uniform_(self.linear1.bias)
    # torch.nn.init.xavier_uniform_(self.linear2.bias)
    # torch.nn.init.xavier_uniform_(self.linear3.bias)
    # torch.nn.init.xavier_uniform_(self.linear4.bias)
    # torch.nn.init.xavier_uniform_(self.linear5.bias)
    # torch.nn.init.xavier_uniform_(self.linear6.bias)
    # torch.nn.init.xavier_uniform_(self.linear7.bias)

  def forward(self, input, act):                                      
    out = act(self.linear1(input))
    out = act(self.linear2(out))
    out = act(self.linear3(out))
    out = act(self.linear4(out))
    out = act(self.linear5(out))
    out = act(self.linear6(out))
    out = F.sigmoid(self.linear7(out))
    out = torch.squeeze(out)
    return out

In [ ]:
nn_num_epochs = 50
nn_input = torch.tensor(train_input, dtype = torch.float32, device = device, requires_grad = True)
print(nn_input[0])
labels = torch.tensor(labels, dtype = torch.float32, device = device, requires_grad = True)
nn_input_test = torch.tensor(test_input, dtype = torch.float32, device = device)

tensor([1.5208e+01, 1.7825e+01, 2.4966e+02, 4.1799e+02, 2.6557e+00, 8.2514e-01,
        1.0929e-02, 2.4562e+02, 1.5208e+01, 1.6464e+01, 6.1617e+01, 7.9728e+01,
        6.3551e+00, 3.5424e+00, 4.0768e-01, 6.1117e+01, 1.6464e+01, 1.7825e+01,
        6.6709e+01, 8.6318e+01, 6.8803e+00, 3.8351e+00, 4.4137e-01, 6.6168e+01,
        6.1617e+01, 6.6709e+01, 2.4966e+02, 3.2304e+02, 2.5749e+01, 1.4353e+01,
        1.6518e+00, 2.4763e+02, 7.9728e+01, 8.6318e+01, 3.2304e+02, 4.1799e+02,
        3.3318e+01, 1.8571e+01, 2.1373e+00, 3.2041e+02, 6.3551e+00, 6.8803e+00,
        2.5749e+01, 3.3318e+01, 2.6557e+00, 1.4803e+00, 1.7037e-01, 2.5540e+01,
        3.5424e+00, 3.8351e+00, 1.4353e+01, 1.8571e+01, 1.4803e+00, 8.2514e-01,
        9.4963e-02, 1.4236e+01, 4.0768e-01, 4.4137e-01, 1.6518e+00, 2.1373e+00,
        1.7037e-01, 9.4963e-02, 1.0929e-02, 1.6384e+00, 6.1117e+01, 6.6168e+01,
        2.4763e+02, 3.2041e+02, 2.5540e+01, 1.4236e+01, 1.6384e+00, 2.4562e+02,
        1.0657e+01], device='cuda:0', gr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
input_dim = len(train_input[0])
print("Input Dimension of NN: " + str(input_dim))
torch.manual_seed(10)
nn_model = NN(input_dim)
nn_model.to(device)
nn_criterion = nn.MSELoss()
nn_optimizer = optim.Adam(nn_model.parameters(), lr = 0.00001, weight_decay=1e-4)

Input Dimension of NN: 73


In [ ]:
def loss_fn(x, y):
    vx = x - torch.mean(x)
    vy = y - torch.mean(y)

    return torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
# x=torch.tensor([1.1,2.1,3])
# y=torch.tensor([1.2,3,1])
# print(loss_fn(x,y).item())

In [ ]:
def train_nn(nn_model, input):
    nn_model.train()
    nn_optimizer.zero_grad()
    output = nn_model(input, F.tanh)                       
    loss = nn_criterion(labels, output)
    loss.backward()
    nn_optimizer.step()
    return loss.item()

In [ ]:
def test_nn(nn_model, input):
    nn_model.eval()
    with torch.no_grad():
        output = nn_model(input, F.tanh)
        return output

In [ ]:
best_pearson_r = 0.0

In [ ]:
def calculate_metrics(y, y_hat, best_pearson_r):
    vx = y.astype(float)
    vy = y_hat.astype(float)
    pearsonR = np.corrcoef(vx, vy)[0, 1]
    spearmanRho = stats.spearmanr(vx, vy)
    MSE = np.mean((vx - vy) ** 2)
    MAE = np.mean(np.absolute(vx - vy))
    RSquared = (pearsonR ** 2)
    if pearsonR > best_pearson_r:
        best_pearson_r = pearsonR

    print("Pearson's R: {}".format(pearsonR))
    print("Spearman's rho: {}".format(spearmanRho))
    print("R Squared: {}".format(RSquared))
    print("MSE: {}".format(MSE))
    print("MAE: {}".format(MAE))
    return best_pearson_r

In [ ]:
for epoch in range(nn_num_epochs):
    nn_train_loss = train_nn(nn_model, nn_input)
    print("Epoch {} : {}".format(epoch + 1, nn_train_loss))
    output = test_nn(nn_model, nn_input_test)
    best_pearson_r = calculate_metrics(np.array(test_labels), np.array(output.tolist()), best_pearson_r)
print("--------------Best Pearson's R: " + str(best_pearson_r))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 1 : 0.07379449903964996
Pearson's R: 0.18132580716523183
Spearman's rho: SpearmanrResult(correlation=0.131747686380055, pvalue=6.291628476289343e-05)
R Squared: 0.03287904834412284
MSE: 0.05963851699841463
MAE: 0.2210437233461746
Epoch 2 : 0.05893866717815399
Pearson's R: 0.18453929200228492
Spearman's rho: SpearmanrResult(correlation=0.13044530924975192, pvalue=7.441897866719602e-05)
R Squared: 0.03405475029270458
MSE: 0.05115869330832369
MAE: 0.20336413081482124
Epoch 3 : 0.050695501267910004
Pearson's R: 0.18004984570022917
Spearman's rho: SpearmanrResult(correlation=0.12686328378803222, pvalue=0.00011715292157699144)
R Squared: 0.03241794693667633
MSE: 0.0467750252531445
MAE: 0.19370124385920254
Epoch 4 : 0.04641861096024513
Pearson's R: 0.169657101704909
Spearman's rho: SpearmanrResult(correlation=0.12515398205279277, pvalue=0.00014487515811969484)
R Squared: 0.028783532158909834
MSE: 0.044354292273631246
MAE: 0.18822132838910197
Epoch 5 : 0.04405515640974045
Pearson's R: 0.

In [ ]:
x= torch.tensor([[1,2,3,4], [2,3,4,5]])
print(x[:,1:3])

tensor([[2, 3],
        [3, 4]])
